#  **EDA por Disciplina**
Este notebook apresenta uma análise exploratória detalhada da base de dados referente aos estudantes do ensino médio matriculados uma disciplina (Português ou Matemática) em duas escolas de portugal. O objetivo principal é identificar padrões e fatores que influenciam o desempenho acadêmico dos estudantes.

## Fonte dos Dados

Os dados utilizados nesta análise são provenientes do *UCI Machine Learning Repository*, especificamente do conjunto de dados "*Student Performance*". Este *dataset* contém informações sobre o desempenho acadêmico de estudantes do ensino médio em Portugal, incluindo notas em português e diversos fatores socioeconômicos.

**Fonte Original**: 

***UCI Machine Learning Repository - Student Performance Dataset***: 
*https://archive.ics.uci.edu/dataset/320/student+performance*

## Objetivos Gerais

A análise realizada visa:

1. Compreender o perfil dos alunos e suas características predominantes
2. Investigar fatores que influenciam o desempenho acadêmico  
3. Fornecer *insights* para intervenções educacionais baseadas em evidências

###### ajustar o path

In [ ]:
# HIDE
import sys
import pathlib

path = pathlib.Path().resolve()
while path.name != 'student_perfomance_tcc' and path != path.parent:
    path = path.parent

# Adicionar a raiz ao sys.path para importar o módulo
if str(path) not in sys.path:
    sys.path.append(str(path))

from ajustar_path import adicionar_modulos_ao_path

# Adiciona a pasta 'modulos' ao path
adicionar_modulos_ao_path()

## 1. Bibliotecas, Módulos e Configurações Iniciais

In [ ]:
#Importação dos Módulos e Funções Desenvolvidos


from pre_modelagem import importar_base
from eda_functions import (
    plot_distribuicao_quantitativas, custom_heatmap,
    resumir_outliers, selecao_impacto_variaveis_categoricas,     
    identificar_extremos_comparaveis,plot_top_diferencas_extremos, 
    plot_notas_faltas 
)

from feature_selection import add_features_describe_pd,avaliacao_variacao_pontuacao_media_por_categoria


import pandas as pd
import numpy as np

# Configurações do Pandas para exibir todas as linhas e colunas no DataFrame 
# para inspeção detalhada dos dados no Jupyter Notebook

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings

# Ignora todos os warnings
warnings.filterwarnings('ignore')

# 1. Visão geral e preparação dos dados

In [ ]:
# Variáveis Globais
materia = "portugues" #ou matemática
cores = 'azul' #ou verde

#### Conhecendo a Base

In [ ]:
df = importar_base(materia)

#Amostra do Data Frame

df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
#Checagem de duplicatas
df.duplicated().sum()

#### **Observações Iniciais**

> - 649 alunos matriculados em português
> - Não a presença de dados faltantes ou linhas duplicadas na base
> - Dtypes: int e str 

#### Variáveis Locais

###### Agrupamento de Estatísticas

In [ ]:
group_stats = {
    "Tendência Central": ["Média", "Mediana (50%)", "Moda"],
    "Dispersão": ["Desvio Padrão", "Coeficiente de Variação (CV)"],
    "Distribuição / Posição": ["Mínimo", "1º Quartil (25%)", "Mediana (50%)", "3º Quartil (75%)", "Máximo"],
    "Normalidade": ["Shapiro-Wilk (p-valor)"],
}


###### Notas

In [ ]:
notas = ['nota1', 'nota2', 'nota_final']

**Categorização de Atributos**

- As variáveis foram categorizadas em três grupos principais:

    >    1. **Variáveis Categóricas Nominais**: Atributos qualitativos sem ordem natural
    >    2. **Variáveis Categóricas Ordinais**: Atributos qualitativos com ordem natural
    >    3. **Variáveis Numéricas**: Atributos quantitativos discretos

In [ ]:
variaveis_categoricas_nominais = [
    'escola', 
    'genero', 
    'endereco', 
    'status_parental', 
    'profissao_mae', 
    'profissao_pai', 
    'motivo_escolha_escola', 
    'responsavel_legal', 
    'apoio_escolar', 
    'apoio_familiar', 
    'aulas_particulares', 
    'atividades_extracurriculares', 
    'frequentou_creche', 
    'interesse_ensino_superior', 
    'acesso_internet', 
    'relacionamento_romantico',
    'tamanho_familia',

    ]

variaveis_categoricas_ordinais = [
    'escolaridade_mae', 
    'escolaridade_pai', 
    'relacao_familiar',
    'tempo_livre',      
    'frequencia_saidas',
    'saude',            
    'tempo_estudo',     
    'tempo_transporte',
    'alcool_dias_uteis',
    'alcool_fim_semana',
    'reprovacoes' 

    ]

variaveis_categoricas = variaveis_categoricas_nominais + variaveis_categoricas_ordinais

variaveis_quantitativas = [
    'idade', 
    'faltas', 
    'nota1', 
    'nota2', 
    'nota_final'
]

# 2. Variáveis Quantitativas

In [ ]:
desc_q = add_features_describe_pd(df, variaveis_quantitativas)

## 2.1 Estatísticas e Distribuições

### 2.1.1 Medidas de Tendencia Central e Dispersão

In [ ]:
mtcp = desc_q[group_stats["Tendência Central"]+group_stats["Dispersão"]].round(4)
mtcp

### 2.1.2 Distribuições, Normalidade e presença de outliers

#### **Estatísticas**

In [ ]:
stats_mddn =desc_q[group_stats["Distribuição / Posição"]+group_stats["Normalidade"]]
stats_mddn

##### **Presença de outliers**

In [ ]:
#resumir outliers 

resumo_outliers = resumir_outliers(df = df[variaveis_quantitativas])


> ### **Resumo Estatístico — Variáveis Quantitativas (Português)**
>
> - **Idade**: Distribuição concentrada (CV = 7,3%), com moda e mediana em 17 anos. Não segue normalidade.
> - **Faltas**: Alta dispersão (CV > 120%), moda zero e forte assimetria. Muitos alunos faltam pouco, mas há outliers com até 32 faltas.
> - **Notas (1, 2, Final)**: Distribuição levemente assimétrica, com variabilidade crescente da nota 1 à nota final. CV entre 24% e 27%.
> - **Todas as variáveis rejeitam a normalidade** pelo teste de Shapiro-Wilk (p < 0.05).
> - **Outliers:** Há a presença de outliers em todas as variáveis quantitativas.


#### **Visualizações**

**Distribuição das Notas**

In [ ]:
plot_distribuicao_quantitativas(df,
                                colunas=notas,
                                mostrar_media=True,
                                paleta=cores,
                                materia=materia)
                                

In [ ]:
plot_distribuicao_quantitativas(df,
                                colunas=notas,
                                mostrar_media=True,
                                paleta=cores,
                                materia=materia,
                                modo = 'hist')
                                

>### **Avaliação das Imagens - Notas (Português)**
> A partir das imagens acima pôde-se observar:
>
> - A assimetria leve à esquerda é visível: as caudas inferiores são mais extensas nas três notas.
>
> - Os outliers inferiores são visíveis em todas as avaliações, com destaque na nota final, que não tem outliers superiores.
>
> - Nota Final mostra um deslocamento ainda mais acentuado da moda para valores abaixo da média.
>
> - A curva de densidade suavizada reforça que a maior parte dos alunos se concentra em torno da média (entre 10 e 13), com cauda inferior mais longa.

**Distribuição de Idade e Faltas**

In [ ]:
plot_distribuicao_quantitativas(df,
                                colunas=['idade','faltas'],
                                mostrar_media=True,
                                paleta=cores,
                                materia=materia)

>### **Avaliação das Imagens - Faltas e Idade (Português)**
>
> - Idade concentrada entre 16–18 anos → variável estável.
> - Apenas 1 outlier (22 anos) → pode ser ignorado ou usado como controle.
> - Faltas com assimetria acentuada à direita → minoria com comportamento extremo.
> - Outliers > 15 faltas indicam possíveis casos de evasão parcial ou desengajamento.


## 2.2 Correlações e Relações

In [ ]:
corr_quant = df[variaveis_quantitativas].corr(method= 'spearman')


In [ ]:
corr_quant

In [ ]:
custom_heatmap(matriz_corr = corr_quant,
               cores = cores,
               n_arq ='heatmap_qualitativas',
               titulo = 'Correlação de Spearman \n Variáveis Quantitativas' ,
               disciplina=materia)

In [ ]:
plot_notas_faltas(df,cor = cores, dir = 'relacao_bivariada' , mat=materia,)

> ### **Correlação e Relações entre Quantitativas — Português**
>
>## Resumo das Observações:
>
> - forte associação entre as notas, especialmente entre **nota2 e nota_final (ρ = 0.94)**.
> - As correlações com faltas são ***fracas e negativas***, sendo mais pronunciada com nota_final (-0.16), indicando 
>   possível impacto de faltas sobre o desempenho.
> - A idade mostra ***correlações próximas de zero***, podendo indicar uma neutralidade como fator explicativo.
> - Os scatterplots confirmam visualmente esses padrões, com ***tendência linear crescente*** entre as notas e 
>   ***ligeira tendência decrescente***(não linear) entre faltas e nota final.

# 3. Variáveis Categóricas e Desempenho Estudantil

## 3.1 Variáveis Categóricas Ordinais

In [ ]:
desc_ord = add_features_describe_pd(df,variaveis_categoricas_ordinais,shapiro_values= False)

In [ ]:
desc_ord

In [ ]:
desc_ord_freq =add_features_describe_pd(df,variaveis_categoricas_ordinais,estudo_frequencia=True)
desc_ord_freq

### 3.1.1 Correlações

In [ ]:
corr_ord = df[variaveis_categoricas_ordinais].corr(method= 'spearman')


In [ ]:
corr_ord

In [ ]:
custom_heatmap(matriz_corr = corr_ord,
               cores = cores,
               n_arq ='heatmap_ordinais',
               titulo = 'Correlação de Spearman \n Variáveis Categóricas Ordinais' ,
               disciplina=materia)

## 3.2 Variáveis Categóricas Nominais

In [ ]:
desc_nom =add_features_describe_pd(df,variaveis_categoricas_nominais,estudo_frequencia=True)
desc_nom

## 3.3 Relação Categorica com Desempenho Escolar


### 3.3.1 Distribuição por Categoria

##### **Nota 1**

In [ ]:
selecao_impacto_variaveis_categoricas(df, 
                                    variaveis_categoricas,
                                    materia= materia,
                                    paleta=cores,
                                    coluna_avaliada='nota1'
                                    )

##### **Nota 2**

In [ ]:
selecao_impacto_variaveis_categoricas(df, 
                                    variaveis_categoricas,
                                    materia= materia,
                                    paleta=cores,
                                    coluna_avaliada='nota2')

##### **Nota Final**

In [ ]:
selecao_impacto_variaveis_categoricas(df, 
                                    variaveis_categoricas,
                                    materia= materia,
                                    paleta=cores,
                                    coluna_avaliada='nota_final')

##### **Faltas**

In [ ]:
selecao_impacto_variaveis_categoricas(df, 
                                    variaveis_categoricas,
                                    materia= materia,
                                    paleta=cores,
                                    coluna_avaliada='faltas')

### 3.3.2 Correlação entre Variáveis Ordinais e Notas e Faltas

In [ ]:
corr_ord = df[variaveis_categoricas_ordinais+notas+['faltas']].corr(method= 'spearman')

In [ ]:
corr_long = corr_ord.where(np.tril(np.ones(corr_ord.shape), k=-1).astype(bool)).stack().reset_index()
corr_long.columns = ['Notas e Faltas','Categoricas Ordinais', 'Correlação']

In [ ]:
corr_ord_n = corr_long[corr_long['Notas e Faltas'].isin(['nota1', 'nota2', 'nota_final','faltas'])]

heatmap_df = corr_ord_n.pivot(index='Categoricas Ordinais', columns='Notas e Faltas', values='Correlação')
heatmap_df.dropna(inplace=True)

In [ ]:
heatmap_df

In [ ]:
custom_heatmap(matriz_corr = heatmap_df,
               cores = cores,
               n_arq ='heatmap_ordinais',
              titulo = 'Correlação de Spearman' ,
               disciplina=materia)

### 3.3.2 Avaliação de Variáveis Categóricas com Base na Variação de Desempenho


#### Metodologia



*Para criação das tabelas da seção calculo-se um índice composto de perfilamento, combinando critérios de distribuição, diversidade e impacto na variável de interesse (nota final ou faltas). Os critérios avaliados foram:*

1. Frequência mínima absoluta e relativa:
A variável deve apresentar frequência mínima por categoria, ajustada ao tamanho da amostra e ao número total de categorias. Isso evita categorias muito raras.

2. Entropia de Shannon (diversidade):
Mede o grau de equilíbrio entre as categorias. Quanto mais equilibradas, maior a entropia e, potencialmente, maior a capacidade da variável de discriminar padrões.

3. Gap de desempenho entre categorias:
Representa a diferença entre as médias da variável de interesse (ex.: nota final) entre as categorias da variável avaliada. Quanto maior essa diferença, maior a capacidade da variável em segmentar o desempenho.

5. Score de Perfil Composto:
Índice final é calculado como a média ponderada da entropia normalizada e do gap de desempenho, permitindo ordenar as variáveis mais informativas.

6. Alertas de dispersão:
Variáveis com alta dispersão (> 80%) e muitas categorias são sinalizadas, pois podem ser menos interpretáveis ou sofrer com sparsidade em modelos.

Este método permite filtrar automaticamente variáveis nominais relevantes para auxiliar na seleção de atributos qualitativos de interesse.

#### **Desempenho - Notas**

##### **Nota 1**

In [ ]:
avaliacao_variacao_pontuacao_media_por_categoria(df,variaveis_categoricas,coluna_avaliada='nota1')

##### **Nota 2**

In [ ]:
avaliacao_variacao_pontuacao_media_por_categoria(df,variaveis_categoricas,coluna_avaliada='nota2')

##### **Nota Final**

In [ ]:
avaliacao_variacao_pontuacao_media_por_categoria(df,variaveis_categoricas)

##### **Média das 3 Notas** 

In [ ]:
df_copy = df.copy()
df_copy['media_notas'] = (df_copy['nota1'] + df_copy['nota2'] + df_copy['nota_final'])/3

avaliacao_variacao_pontuacao_media_por_categoria(df_copy,variaveis_categoricas,coluna_avaliada='media_notas')

**Faltas**

In [ ]:
avaliacao_variacao_pontuacao_media_por_categoria(df,variaveis_categoricas,coluna_avaliada='faltas')

# 4. Análise de Outliers e Perfil de Grupos

## 4.1 Resumo de Outiliers

### 4.1.1 Resumo Descritivo dos Dados Avaliados

> Os outliers foram identificados usando o método do Intervalo Interquartil (IQR):
>   - Limite Inferior = Q1 - 1.5 * IQR
>   - Limite Superior = Q3 + 1.5 * IQR

In [ ]:
# Converte valores numéricos armazenados como strings com vírgulas para floats
resumo_outliers_numerico = resumo_outliers.applymap(
    lambda x: float(str(x).replace(",", ".")) if isinstance(x, str) and "%" not in x else x
)

### 4.1.2 Perfil dos Outliers



##### **Levantamento de recursos para a análise**

In [ ]:
variaveis_de_interesse = variaveis_categoricas + ['idade']

In [ ]:
# Lista de variáveis com outliers
outliers_index_list = resumo_outliers_numerico.index.tolist()

In [ ]:
limites_outliers = {}

for var in outliers_index_list:
    out_baixo = resumo_outliers_numerico.loc[var, 'Outliers < L1']
    out_alto = resumo_outliers_numerico.loc[var, 'Outliers > L3']

    if out_baixo > 0 or out_alto > 0:
        limites_outliers[var] = {
            'lim_inf': float(resumo_outliers_numerico.loc[var, 'Limite Inferior (L1)']),
            'lim_sup': float(resumo_outliers_numerico.loc[var, 'Limite Superior (L3)'])
        }


**Notas**

##### **Perfil Outiliers - Nota1**

###### **Nota 1 - Limite Inferior**

In [ ]:
var = 'nota1'
perfil = df[df[var] < limites_outliers[var]['lim_inf']]

perfil = add_features_describe_pd(df=perfil,colunas=variaveis_de_interesse,estudo_frequencia=True)

perfil

###### **Nota 1 - Limite Superior**

In [ ]:
var = 'nota1'
perfil = df[df[var] > limites_outliers[var]['lim_sup']]

perfil = add_features_describe_pd(df=perfil,colunas=variaveis_de_interesse,estudo_frequencia=True)

perfil

##### **Perfil Outiliers - Nota2**

###### **Nota 2 - Limite Inferior**

In [ ]:
var = 'nota2'

perfil = df[df[var] < limites_outliers[var]['lim_inf']]
      
perfil = add_features_describe_pd(df= perfil
                                ,colunas=variaveis_de_interesse,
                                estudo_frequencia=True)

perfil

###### **Nota 2 - Limite Superior**

In [ ]:
var = 'nota2'
perfil = df[df[var] > limites_outliers[var]['lim_sup']]

perfil = add_features_describe_pd(df=perfil,colunas=variaveis_de_interesse,estudo_frequencia=True)

perfil

##### **Perfil Outiliers - Nota Final**

###### **Nota Final - Limite Inferior**

In [ ]:
var = 'nota_final'
perfil = df[df[var] < limites_outliers[var]['lim_inf']]

perfil = add_features_describe_pd(df=perfil,colunas=variaveis_de_interesse,estudo_frequencia=True)

perfil

**Faltas**

##### **Perfil Outiliers - Faltas**

###### **Faltas - Limite Superior**

In [ ]:
var = 'faltas'
perfil = df[df[var] > limites_outliers[var]['lim_sup']]

perfil = add_features_describe_pd(df=perfil,colunas=variaveis_de_interesse,estudo_frequencia=True)

perfil

## 4.4 Identificação de Diferenças de Perfil entre Grupos de Desempenho - Estudo com Base na Frequência de Categorias

In [ ]:
df_dif, n_baixo, n_alto, q1_lim, q3_lim = identificar_extremos_comparaveis(
    df, 'nota_final', variaveis_categoricas
)

In [ ]:
plot_top_diferencas_extremos(
    df_diferencas=df_dif,
    materia='portugues',  
    q1_lim=q1_lim,
    q3_lim=q3_lim,
    n_baixo=n_baixo,
    n_alto=n_alto,
    top_n=10,  
    diretorio='graficos_diferencas_perfil',
    salvar=True
)

## 4.5 Identificação de Diferenças de Perfil entre Grupos - Aprovados vs Reprovados

In [ ]:
resumo = df.groupby('aprovacao').agg(Contagem=('faltas', 'count'))
resumo['Percentual (%)'] = (resumo['Contagem'] / resumo['Contagem'].sum() * 100).round(2)
resumo.reset_index()



In [ ]:
resumo = df.groupby('aprovacao').agg({
    'nota1': ['mean', 'std', 'min', 'max'],
    'nota2': ['mean', 'std', 'min', 'max'],
    'nota_final': ['mean', 'std', 'min', 'max']
})


In [ ]:


resumo.index.name = None 

# Traduz estatísticas
traducao = {'mean': 'Média', 'std': 'Desvio', 'min': 'Mínimo', 'max': 'Máximo'}
resumo.columns = pd.MultiIndex.from_tuples(
    [(col[0], traducao[col[1]]) for col in resumo.columns],
)

# Arredonda
resumo = resumo.round(2)
# Identifique as colunas que marcam os blocos de cada nota
nota1_cols = [col for col in resumo.columns if col[0] == 'nota1']
nota2_cols = [col for col in resumo.columns if col[0] == 'nota2']
nota_3cols = [col for col in resumo.columns if col[0] == 'nota2']

styles = [
    {'selector': f'th.col{i}', 'props': [('border-left', '2.0px solid #999')]}
    for i, col in enumerate(resumo.columns)
    if col in [nota1_cols[-1], nota2_cols[-1],nota_3cols[-1]] 
]
resumo_style = resumo.style.set_table_styles(styles + [
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
]).set_caption("Resumo estatístico das notas por grupo de aprovação")

In [ ]:
resumo_style 

In [ ]:
aprovados = df[df['aprovacao']=='Aprovado']
reprovados = df[df['aprovacao']=='Reprovado']


##### Perfilamento Aprovados

In [ ]:
avaliacao_variacao_pontuacao_media_por_categoria(aprovados,variaveis_categoricas,coluna_avaliada='nota_final')

##### Perfilamento Reprovados

In [ ]:
avaliacao_variacao_pontuacao_media_por_categoria(reprovados,variaveis_categoricas,coluna_avaliada='nota_final')